# Pipeline 

## Its not an Algorithm its a technquie that together multiple step and uses ouput of previous as input to other. Pipelien is better becasue it apply the same preprocessing o both train and test 

## Column Transformer split the data and apply transformation column wise whereas in Pipeline all data is treated as one.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df= pd.read_csv('titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df= df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [5]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [6]:

ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

In [8]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [9]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [10]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

In [12]:
import pickle 

In [13]:
#Saving the class and Model

pickle.dump(ohe_sex,open('ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl', 'wb'))
pickle.dump(clf,open('clf.pkl', 'wb'))

In [14]:
#Decosing the saved model

ohe_sex= pickle.load(open('ohe_sex.pkl', 'rb'))
ohe_embarked= pickle.load(open('ohe_embarked.pkl', 'rb'))
clf= pickle.load(open('clf.pkl', 'rb'))

In [15]:
test_input= np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype= object).reshape(1,-1)

In [16]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [18]:
test_input_sex= ohe_sex.transform(test_input[:,1].reshape(1,-1))

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
test_input_sex

array([[0., 1.]])

In [20]:
test_input_embarked= ohe_embarked.transform(test_input[:,-1].reshape(1,-1))

In [21]:
test_input_embarked

array([[0., 0., 1.]])

In [22]:
test_input_age= test_input[:,2].reshape(-1,1)

In [23]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [24]:
test_input_transformed.reshape(-1,1)

array([[2],
       [0],
       [0],
       [10.5],
       [31.0],
       [0.0],
       [1.0],
       [0.0],
       [0.0],
       [1.0]], dtype=object)

In [25]:
clf.predict(test_input_transformed)

array([1], dtype=int64)

## We need to train the data by doing preprocessing but when we work to predict same preprocessing need to be carry out on the Input data online data need to be preprocessed and all the training steps need to be carry out thats why we use pipeline where once train data processed throgh pipeline dnt need to repeat on test data again.

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [27]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [28]:
#Handling missing value

trf1= ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),     #we have used number instead of the name because the ouput from the pipeline will be numpy array and they dont ahve columnname and if u enter the column name when teh output from one column enter other pipeline it start getting error
    ('impute_embarked',SimpleImputer(strategy= 'most_frequent'),[6]) #number will act same as dataframne on numpy array too so that why we use no and not the name as name would have gone once it exit from the piepline
],remainder='passthrough')

In [29]:
#Handling Categorical data

trf2= ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse= False, handle_unknown= 'ignore'),[1,6]) #as everything is happning inside the and not new numpy is created so we dont need to do seprately 
],remainder= 'passthrough') #we dont apply first column to drop because its a decision tree and we will loss data DT are capable of handling the dummy data but when in linear model we need to use the drop option as it will creat confusion 

In [30]:
# Scaling

trf3= ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))   #as using feature selection we need to use MinMax its a rule and requirmnet if 
                                            #not using feature selection we can suer StandardScler fun
])                                          #10 becasue w have initially 7 column out of which 2 got one hot encoding sex 
                                            #and embarked that gave 2+3 as first column is not droped and from initial stage we have 5 so 5+2+3=10

In [31]:
#feature Extraction
trf4= SelectKBest(score_func= chi2,k=8)

In [32]:
trf5= DecisionTreeClassifier()

In [33]:
pipe= Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [34]:
pipe= make_pipeline(trf1,trf2,trf3,trf4,trf5)


#Pipelien nad makepipeline work the same way just differnce in writting also have same differnce in ColumnTranformer 
#and make_column_transformer(only last 2 need to be entered)

In [35]:
from sklearn import set_config
set_config(display='diagram')  #it not necessary but if the pipeline digram is not seen writ the code 

pipe.fit(X_train,y_train)
#suppose we are not using the model and pipe is ready we need to sue Fit_transform to preprocess but 
# as model need to be train use fit so that can predict

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002D84D985630>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [36]:
y_pred= pipe.predict(X_test)

In [42]:
print(pipe.named_steps)#['trf1'].transformers_

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])]), 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1, 6])]), 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]), 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000002D84D985630>), 'decisiontreeclassifier': DecisionTreeClassifier()}


In [43]:
pipe.named_steps['columntransformer-1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [44]:
pipe.named_steps['columntransformer-1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [45]:
pipe.named_steps['columntransformer-1'].transformers_[0][1]

SimpleImputer()

In [46]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_  #got mena what the colum took

array([29.49884615])

In [47]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_ #getting the most revert data

array(['S'], dtype=object)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)  #low as features are dropped rework and check

0.6256983240223464

# Cross Validation

In [52]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring= 'accuracy').mean()

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in vers

0.6391214419383433

# Gridsearch in Pipeline

In [58]:
params= {
    'decisiontreeclassifier__max_depth': [1,2,3,4,5,None]  #need to say trf5 becasue need to aply the tunning on that pipe area _ _
}

In [59]:
from sklearn.model_selection import GridSearchCV
grid= GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in vers

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unkno...ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000002D84D985630>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [61]:
grid.best_score_

0.6391214419383433

In [63]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

In [64]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [65]:
pipe= pickle.load(open('pipe.pkl', 'rb'))

In [66]:
pipe.predict(test_input)

C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\shubh\anaconda3\envs\env\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)